# pySEAFOM Dynamic Range — Hilbert & THD

**Repository**: SEAFOM-Fiber-Optic-Monitoring-Group/pySEAFOM

This notebook runs the dynamic range analysis calculation functionality in the pySEAFOM library using synthetic or real DAS data.

## Overview

This notebook:
- Loads a 2D DAS matrix `.npy`
- Extracts a 1D trace in a spatial window around `POS_M`
- Optionally converts phase (rad) → strain (µε) and applies a high‑pass filter
- Calculates dynamic range limit using **Hilbert** (envelope mismatch vs theoretical ramp)
- Calculates dynamic range limit using **THD** (sliding THD threshold)

## Key outputs (per run)

- Trigger time (absolute) `[s]` and **Δt from window start** `[s]`
- **Peak strain (last cycle before distortion)** `[µε]`
- Optional **Peak/Basis in rdB re radian/√Hz**  
  Computed only if `RADIAN_BASIS` and `GAUGE_LENGTH_M` are available.
- Optional plots (Hilbert/THD) and CSV summary logs (one row per run)


## 1. Setup and Imports

Import `pySEAFOM.dynamic_range` and basic Python utilities.


In [ ]:
# Imports
import numpy as np

from pySEAFOM import dynamic_range


## 2. Configure the Test Parameters

### Control
- `ANALYSIS_TYPE`: `"hilbert"`, `"thd"`, or `"both"`
- `SAVE_RESULTS`: if `True`, saves plots + CSV to `OUTPUT_DIR`
- `SHOW_PLOTS`: if `True`, displays figures

### Data extraction
DAS data is expected as a 2D `.npy` matrix with shape `(time, space)`.


In [ ]:
# ======================================================================
# CONTROL
# ======================================================================
ANALYSIS_TYPE = "both"          # "hilbert", "thd", or "both"
SAVE_RESULTS = True             # Saves figures + CSV files in OUTPUT_DIR
OUTPUT_DIR = "results_dynamic_range"
SHOW_PLOTS = True               # Display figures (plt.show)
PLOT_METADATA_BOX = True        # Add measurement-parameter box to the plot

# ======================================================================
# INPUT PARAMETERS (data extraction)
# ======================================================================
FOLDER_OR_FILE = "teste"        # Folder with .npy files OR a single .npy file
REPETITION_RATE_HZ = 2500.0     # Sampling / interrogator rate [Hz]
DELTA_X_M = 0.2                 # Spatial step between channels [m]

X1_M = 100.0                    # Spatial window start [m]
X2_M = 300.0                    # Spatial window end [m]
POS_M = 200.0                   # Central position inside the spatial window [m]

TIME_START_S = 35.0             # Analysis window start time [s]
DURATION_S = 30.0               # Analysis window duration [s]

AVERAGE_OVER_COLS = 5           # Average N adjacent channels starting at POS_M
DATA_IS_STRAIN = False          # True if loaded trace is already strain [µε]
GAUGE_LENGTH_M = 6.38           # Gauge length used by interrogator [m]
HIGHPASS_HZ = 5.0               # High-pass cutoff [Hz] (set None to disable)

# Optional: only used to compute rdB re radian/√Hz
RADIAN_BASIS = 2.0              # Radians at noise level for self-noise (set None to disable)

# ======================================================================
# COMMON PARAMETER (used by both Hilbert and THD)
# ======================================================================
SAFEZONE_S = 1.0                # Initial safe zone where triggering is ignored [s]

# ======================================================================
# HILBERT PARAMETERS
# ======================================================================
MAX_STRAIN_UE = 0.51            # Final theoretical envelope amplitude [µε]
REF_FREQ_HZ = 50.0              # Expected sine frequency [Hz]
HILBERT_SMOOTH_WINDOW_S = 0.5   # Envelope smoothing window [s]
HILBERT_ERROR_THRESHOLD_FRAC = 0.3   # Relative error threshold (0.3 = 30%)

# ======================================================================
# THD PARAMETERS
# ======================================================================
THD_WINDOW_S = 1.0              # Sliding window length [s]
THD_OVERLAP = 0.75              # Window overlap fraction (0.75 = 75%)
THD_THRESHOLD_FRAC = 0.15       # THD threshold (0.15 = 15%)
THD_MEDIAN_WINDOW_S = 0.005     # Median filter window applied to THD curve [s]
MIN_TRIGGER_DURATION_S = 1.0    # Minimum continuous violation time to trigger [s]


## 3. Load DAS Data and Extract a 1D Trace

This step loads a `.npy` matrix (or concatenates multiple `.npy` files from a folder),
then extracts a 1D trace around `POS_M` within `[X1_M, X2_M]`.

This step can also use a simulated matrix available on simulator_dynamic_range.py on the "source" folder



In [ ]:
time_s, trace_raw = dynamic_range.load_dynamic_range_data(
    folder_or_file=FOLDER_OR_FILE,
    fs=REPETITION_RATE_HZ,
    delta_x_m=DELTA_X_M,
    x1_m=X1_M,
    x2_m=X2_M,
    test_sections_channels=POS_M,
    time_start_s=TIME_START_S,
    duration=DURATION_S,
    average_over_cols=AVERAGE_OVER_COLS,
    matrix_layout="auto",
)


## 4. Convert to Microstrain (and Optional High‑Pass)

If `DATA_IS_STRAIN=False`, the trace is interpreted as **phase** (rad) and converted to strain (µε)
using the same physics used by `pySEAFOM.dynamic_range.data_processing()`.


In [ ]:
signal_ue = dynamic_range.data_processing(
    trace=trace_raw,
    data_is_strain=DATA_IS_STRAIN,
    gauge_length=GAUGE_LENGTH_M,
    highpass_hz=HIGHPASS_HZ,
    fs=REPETITION_RATE_HZ,
)

## 5. Hilbert Dynamic Range Analysis

Computes the envelope mismatch against a theoretical ramped sinusoid and finds the first time the
relative error exceeds the threshold (after `SAFEZONE_S`).


In [ ]:
if ANALYSIS_TYPE.lower() in {"hilbert", "both"}:
    dynamic_range.analyze_dynamic_range_hilbert(
        # 1) required data
        time_s=time_s,
        signal_microstrain=signal_ue,
        max_strain_microstrain=MAX_STRAIN_UE,

        # 2) analysis parameters
        ref_freq_hz=REF_FREQ_HZ,
        smooth_window_s=HILBERT_SMOOTH_WINDOW_S,
        error_threshold_frac=HILBERT_ERROR_THRESHOLD_FRAC,
        safezone_s=SAFEZONE_S,

        # Optional: dB re radian/√Hz (only computed if radian_basis + gauge_length are available)
        radian_basis=RADIAN_BASIS,
        gauge_length=GAUGE_LENGTH_M,

        # 3) optional metadata / run context (plot box + CSV)
        time_start_s=TIME_START_S,
        duration=DURATION_S,
        folder_or_file=FOLDER_OR_FILE,
        test_sections_channels=POS_M,
        data_is_strain=DATA_IS_STRAIN,
        average_over_cols=AVERAGE_OVER_COLS,

        fs=REPETITION_RATE_HZ,
        delta_x_m=DELTA_X_M,
        highpass_hz=HIGHPASS_HZ,

        # 4) I/O options
        show_plot=SHOW_PLOTS,
        save_results=SAVE_RESULTS,
        results_dir=OUTPUT_DIR,
        plot_metadata_box=PLOT_METADATA_BOX,
    )


## 6. THD Dynamic Range Analysis

Runs a sliding-window THD estimate and finds the first time THD stays above threshold for at least
`MIN_TRIGGER_DURATION_S` (after `SAFEZONE_S`).


In [ ]:
if ANALYSIS_TYPE.lower() in {"thd", "both"}:
    dynamic_range.analyze_dynamic_range_thd(
        # 1) required data
        time_s=time_s,
        signal_microstrain=signal_ue,

        # 2) analysis parameters
        ref_freq_hz=REF_FREQ_HZ,
        window_s=THD_WINDOW_S,
        overlap=THD_OVERLAP,
        thd_threshold_frac=THD_THRESHOLD_FRAC,
        median_window_s=THD_MEDIAN_WINDOW_S,
        min_trigger_duration=MIN_TRIGGER_DURATION_S,
        safezone_s=SAFEZONE_S,

        # Optional: dB re radian/√Hz (only computed if radian_basis + gauge_length are available)
        radian_basis=RADIAN_BASIS,
        gauge_length=GAUGE_LENGTH_M,

        # 3) optional metadata / run context (plot box + CSV)
        time_start_s=TIME_START_S,
        duration=DURATION_S,
        folder_or_file=FOLDER_OR_FILE,
        test_sections_channels=POS_M,
        data_is_strain=DATA_IS_STRAIN,
        average_over_cols=AVERAGE_OVER_COLS,

        fs=REPETITION_RATE_HZ,
        delta_x_m=DELTA_X_M,
        highpass_hz=HIGHPASS_HZ,

        # 4) I/O options
        show_plot=SHOW_PLOTS,
        save_results=SAVE_RESULTS,
        results_dir=OUTPUT_DIR,
        plot_metadata_box=PLOT_METADATA_BOX,
    )


## 7. Minimal Calls (Examples)

Use these if you only want a quick plot/printout without providing full metadata.


In [ ]:
# Minimal THD call (quick run)
# dynamic_range.analyze_dynamic_range_thd(
#     time_s=time_s,
#     signal_microstrain=signal_ue,
#     time_start_s=TIME_START_S,
# )

# Minimal Hilbert call (quick run)
# dynamic_range.analyze_dynamic_range_hilbert(
#     time_s=time_s,
#     signal_microstrain=signal_ue,
#     max_strain_microstrain=MAX_STRAIN_UE,
#     time_start_s=TIME_START_S,
# )


## 8. Output Artifacts

If `SAVE_RESULTS=True`, the following are written to `OUTPUT_DIR`:

- `dynamic_range_hilbert.png` and `dynamic_range_hilbert.csv` (one row per run)
- `dynamic_range_thd.png` and `dynamic_range_thd.csv` (one row per run)

CSV metrics include:
- Trigger time (absolute) `[s]`
- Δt from window start `[s]`
- Peak of last cycle before distortion `[µε]`
- Optional Peak/Basis **dB re radian/√Hz** (only if `RADIAN_BASIS` + `GAUGE_LENGTH_M` are provided)
